In [7]:
from catboost import CatBoostRanker, Pool, MetricVisualizer
from copy import deepcopy
import numpy as np
import os
from sklearn.metrics import ndcg_score
import pandas as pd

In [1]:
!apt -y install zip unzip git 
!pip install catboost scikit-learn pandas sentence-transformers

Reading package lists... Done
Building dependency tree       
Reading state information... Done
git is already the newest version (1:2.17.1-1ubuntu0.17).
The following NEW packages will be installed:
  unzip zip
0 upgraded, 2 newly installed, 0 to remove and 22 not upgraded.
Need to get 335 kB of archives.
After this operation, 1205 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 unzip amd64 6.0-21ubuntu1.2 [168 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/main amd64 zip amd64 3.0-11build1 [167 kB]
Fetched 335 kB in 1s (609 kB/s)m
debconf: delaying package configuration, since apt-utils is not installed

78Selecting previously unselected package unzip.
(Reading database ... 13495 files and directories currently installed.)
Preparing to unpack .../unzip_6.0-21ubuntu1.2_amd64.deb ...
7Progress: [  0%] [..........................................................] 87Progress: [  9%] [#####..................................

In [3]:
!curl --header 'Host: s325vlx.storage.yandex.net' --user-agent 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:109.0) Gecko/20100101 Firefox/111.0' --header 'Accept: text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8' --header 'Accept-Language: en-US,en;q=0.5' --referer 'https://disk.yandex.ru/' --cookie '_yasc=PNhRrlK28sQvfjWTKkGK9TroD92noihW+ut4GsobDisgLSdSCEY1sANHxq/c' --header 'Upgrade-Insecure-Requests: 1' --header 'Sec-Fetch-Dest: iframe' --header 'Sec-Fetch-Mode: navigate' --header 'Sec-Fetch-Site: cross-site' --header 'Sec-GPC: 1' 'https://s325vlx.storage.yandex.net/rdisk/f2555f3dfa884725b885d7c698d9df205423f14defbe4a662f9ad77ca2267cbd/64171c17/ggdlMgCJiwS_4Bg2cqsH7UgFHNNiHGPZ3Q6l8FPvPdMJZHBVK7xZ7NPqGsTCvM4lNGmU3KDGPhay16llqfGoqA==?uid=546929459&filename=dataset.zip&disposition=attachment&hash=&limit=0&content_type=application%2Fzip&owner_uid=546929459&fsize=111141281&hid=8398c9204a04bf26a40e409861906fc9&media_type=compressed&tknv=v2&etag=02fd239026a7e2ff565241c736d09156&rtoken=acDgl9ZxK5am&force_default=yes&ycrid=na-8c97e2e5a8d3f66cce369f9dd4fd0bf2-downloader11f&ts=5f741a05df3c0&s=86cc4dd69dbd29ba63a5292cb0fed77eb533b80ab57019f1aef4e1ece28048f8&pb=U2FsdGVkX1_q84MsDeEEDjG_TKOYR0qoVF5dpXKzRFWi6oMis3RWMm7X_TIQVVeMqQv3WBYI91UvOGXhRLnbz-gZwyrnLdtaQ9thZLq3C2g' --output 'dataset.zip'

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  105M  100  105M    0     0  21.2M      0  0:00:04  0:00:04 --:--:-- 21.2M


In [4]:
!unzip dataset.zip

Archive:  dataset.zip
   creating: dataset/
  inflating: dataset/readme.txt      
  inflating: dataset/ranking_train.jsonl  
  inflating: dataset/ranking_test.jsonl  


In [5]:
import json
with open('dataset/ranking_train.jsonl') as f:
    train_data = [json.loads(line) for line in f]

In [8]:
post_texts = []
comments_texts = []
total_texts = []
scores = []
groups = []
for i,example in enumerate(train_data):
    for comment in example['comments']:
        post_texts.append(example['text'])
        comments_texts.append(comment['text'])
        total_texts.append(example['text'] + ' COMMENT: ' + comment['text'])
        scores.append(comment['score'])
        groups.append(i)
scores = np.array(scores)/4
train_dataframe = pd.DataFrame([post_texts,comments_texts]).T

In [68]:
train = Pool(
    data=train_dataframe[:300000],
    label=1 - scores[:300000],
    group_id=groups[:300000],
    text_features = [0,1]
)

test = Pool(
    data=train_dataframe[300000:],
    label=1 - scores[300000:],
    group_id=groups[300000:],
    text_features = [0,1]
)

In [71]:
default_parameters = {
    'iterations': 5000,
    'custom_metric': ['NDCG'],
    'verbose': True,
    'random_seed': 42,
}

parameters = {}


In [72]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
        
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=False)
    
    return model

In [9]:
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('all-MiniLM-L6-v2')

In [10]:
sentence_embeddings = model.encode(total_texts,show_progress_bar = True)

Batches:   0%|          | 0/13767 [00:00<?, ?it/s]

In [82]:
train = Pool(
    data=sentence_embeddings[:400000],
    label=1.0 - scores[:400000],
    group_id=groups[:400000],
)

test = Pool(
    data=sentence_embeddings[400000:],
    label=1.0 - scores[400000:],
    group_id=groups[400000:],
)

In [83]:
default_parameters = {
    'iterations': 5000,
    'custom_metric': ['NDCG'],
    'verbose': True,
    'random_seed': 0,
}

parameters = {}


In [84]:
def fit_model(loss_function, additional_params=None, train_pool=train, test_pool=test):
    parameters = deepcopy(default_parameters)
    parameters['loss_function'] = loss_function
    parameters['train_dir'] = loss_function
    
    if additional_params is not None:
        parameters.update(additional_params)
    
    model = CatBoostRanker(**parameters)
    model.fit(train_pool, eval_set=test_pool, plot=False)
    
    return model

In [87]:
model = fit_model('YetiRankPairwise', {'task_type': 'GPU'})
ndcg_score(1.0-scores[400000:].reshape(-1,5),model.predict(test).reshape(-1,5))

Default metric period is 5 because PFound, NDCG is/are not implemented for GPU
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric PFound is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	test: 0.8953362	best: 0.8953362 (0)	total: 64.9ms	remaining: 5m 24s
1:	total: 125ms	remaining: 5m 11s
2:	total: 184ms	remaining: 5m 6s
3:	total: 244ms	remaining: 5m 4s
4:	total: 301ms	remaining: 5m
5:	test: 0.9078318	best: 0.9078318 (5)	total: 362ms	remaining: 5m 1s
6:	total: 418ms	remaining: 4m 58s
7:	total: 474ms	remaining: 4m 56s
8:	total: 532ms	remaining: 4m 54s
9:	total: 588ms	remaining: 4m 53s
10:	test: 0.9259665	best: 0.9259665 (10)	total: 649ms	remaining: 4m 54s
11:	total: 707ms	remaining: 4m 53s
12:	total: 763ms	remaining: 4m 52s
13:	total: 820ms	remaining: 4m 52s
14:	total: 877ms	remaining: 4m 51s
15:	test: 0.9279175	best: 0.9279175 (15)	total: 938ms	remaining: 4m 52s
16:	total: 995ms	remaining: 4m 51s
17:	total: 1.05s	remaining: 4m 51s
18:	total: 1.11s	remaining: 4m 50s
19:	total: 1.17s	remaining: 4m 50s
20:	test: 0.9283592	best: 0.9283592 (20)	total: 1.23s	remaining: 4m 50s
21:	total: 1.28s	remaining: 4m 50s
22:	total: 1.34s	remaining: 4m 50s
23:	total: 1.4s	remaining: 4

0.8854483540084387

In [88]:
model_rmse = fit_model('RMSE', {'task_type': 'GPU'})
ndcg_score(1.0-scores[400000:].reshape(-1,5),model_rmse.predict(test).reshape(-1,5))

/opt/conda/lib/python3.10/site-packages/catboost/core.py:6240: RuntimeWarning: Regression loss ('RMSE') ignores an important ranking parameter 'group_id'
  warnings.warn("Regression loss ('{}') ignores an important ranking parameter 'group_id'".format(loss_function), RuntimeWarning)


Learning rate set to 0.074977


Default metric period is 5 because NDCG is/are not implemented for GPU
Metric NDCG:type=Base is not implemented on GPU. Will use CPU for metric computation, this could significantly affect learning time


0:	learn: 0.3514485	test: 0.3514193	best: 0.3514193 (0)	total: 17.2ms	remaining: 1m 26s
1:	total: 31.7ms	remaining: 1m 19s
2:	total: 45.2ms	remaining: 1m 15s
3:	total: 59.8ms	remaining: 1m 14s
4:	total: 73.6ms	remaining: 1m 13s
5:	learn: 0.3444181	test: 0.3443096	best: 0.3443096 (5)	total: 90.8ms	remaining: 1m 15s
6:	total: 104ms	remaining: 1m 13s
7:	total: 118ms	remaining: 1m 13s
8:	total: 133ms	remaining: 1m 13s
9:	total: 147ms	remaining: 1m 13s
10:	learn: 0.3409023	test: 0.3406877	best: 0.3406877 (10)	total: 163ms	remaining: 1m 13s
11:	total: 177ms	remaining: 1m 13s
12:	total: 191ms	remaining: 1m 13s
13:	total: 206ms	remaining: 1m 13s
14:	total: 221ms	remaining: 1m 13s
15:	learn: 0.3391414	test: 0.3389429	best: 0.3389429 (15)	total: 237ms	remaining: 1m 13s
16:	total: 252ms	remaining: 1m 13s
17:	total: 266ms	remaining: 1m 13s
18:	total: 281ms	remaining: 1m 13s
19:	total: 297ms	remaining: 1m 13s
20:	learn: 0.3381948	test: 0.3380445	best: 0.3380445 (20)	total: 314ms	remaining: 1m 14s
2

0.8822114016160718